# N-Queens MCMC P.2
## Estimating the number of solutions.

In [123]:
# Imports
import time, sys, timeit
from IPython.display import clear_output
import numpy as np
import itertools as it
import math

In [124]:
# For reproducibility
np.random.seed(2022)

In [204]:
# Initialisation
N = 27
C = math.comb(N,2)
z0 = np.arange(1,N+1)
beta = 1

idx_pairs = np.array(list(it.combinations(z0,2))) - [1,1]
col_diff = np.array([j-i for (i,j) in idx_pairs])
np.random.shuffle(z0)

In [205]:
def swap(z, i, j):
    """
    Swaps the elements of z at indices i and j, then returns z. Inplace.
    """
    z[[i, j]] = z[j], z[i]
    return z

In [206]:
def threats(z, i):
    """
    Returns number of queens threatening queen i.
    """
    Q = np.delete(np.arange(N), i) # Other queens
    return np.sum(abs(Q-i)==abs(z[Q]-z[i]))

In [207]:
def loss_diff(z, i, j):
    """
    Given a state z and swap operation (i,j), calculates the change in loss.
    """
    old = threats(z,i) + threats(z,j)
    y = swap(z, i, j)
    new = threats(y,i) + threats(y,j)
    z = swap(y, i, j)

    return new - old

In [208]:
### Loss function runs in n(n+1)/2 steps.
def loss(z):
    """
    Interprets z as chessboard with N queens threatening each other diagonally.
    Counts the number of unique pairs of threatening queens.
    """
    # Compute pairwise differences in z.
    row_diff = np.array([abs(z[j]-z[i]) for (i,j) in idx_pairs])
    loss = np.sum(col_diff==row_diff)
    return loss

In [315]:
# Run search
MAX_ITERS=1000000
REG_ITERS = 1000
z = z0.copy()
I = np.append(idx_pairs, [[0, 0]],axis=0)
l = loss(z)
for t in range(1, MAX_ITERS):

    # Calculate loss
    if t % REG_ITERS == 0:
        
        # Print current loss
        clear_output(wait = True)
        print("t =", t, "| conflicts =", l)

    # Choose a random swap
    beta = 7 # np.log(t**2/N)
    i, j = idx_pairs[np.random.choice(C, size=1)][0]
    diff = loss_diff(z,i,j)
    acc = min(1, np.exp(-beta*diff))
    if np.random.rand() < acc:
        z = swap(z,i,j)
        l += diff
    
    # If a solution is found, exit.
    if l <= 0:
        break
    
if (loss(z) == 0):    
    print("Here's a valid solution: ", z, "\nFound after ", t, " steps. (beta = ", beta, ")")

Here's a valid solution:  [22  6  2 18 20 11  3 19  4 10 27 14  1 15 24  8 23 26 17 13  7 25 12 21
  9 16  5] 
Found after  485  steps. (beta =  7 )


In [330]:
# Estimating the number of solutions.
beta = 7
T = 20
B = np.linspace(1, beta, T)
print(B)
M = 100
Z_ratios = []
THRESH = 1000

for t in range(T-1):
    
    # Print current length of L
    clear_output(wait = True)
    print(t)
    
    # Initial random sample
    x = np.arange(1,N+1)
    np.random.shuffle(x)
    
    # Loss records
    l = loss(x)
    L  = np.array([l])
    for k in range(1, MAX_ITERS):    
        
        i, j = idx_pairs[np.random.choice(C, size=1)][0]
        diff = loss_diff(x,i,j)
        acc = min(1, np.exp(-B[t]*diff))
        if np.random.rand() < acc:
            x = swap(x,i,j)
            l += diff
        
        if (k > THRESH):
            L = np.append(L, l)    

        if len(L) == M: break
    Z_ratios = np.append(Z_ratios, np.mean(np.exp(-(B[t+1] - B[t]) * L)))

Z_inf_ratio = np.prod(Z_ratios)
num_solutions = Z_inf_ratio * np.math.factorial(N)

18


In [332]:
print("estimated # of solutions (log method) = ", num_solutions)
print("Magnitude = ", np.log10(num_solutions))

estimated # of solutions (log method) =  8.649962751386016e+21
Magnitude =  21.937014237302694


In [264]:
np.log10(234_907_967_154_122_528)

17.370897746587715